In [ ]:
# Run just to test if ollama is running properly

from llama_index.llms.ollama import Ollama

llm = Ollama(model="mistral:latest", request_timeout=1000)

response = llm.complete("What is the history of LEGO?")

print(response)

In [1]:
from pathlib import Path

import qdrant_client
from llama_index.core import (
    VectorStoreIndex,
    ServiceContext,
    SimpleDirectoryReader
)
from llama_index.llms.ollama import Ollama
from llama_index.core.storage.storage_context import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore

In [2]:
# Path to your data folder
documents = SimpleDirectoryReader("../data2").load_data()

In [3]:
# Initialize the vector store
client = qdrant_client.QdrantClient(
    path="../qdrant_data"
)
vector_store = QdrantVectorStore(client=client, collection_name="data2")
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [6]:
# Service Context seems to be deprecated, will switch soon to new setting
llm = Ollama(model="mistral:latest", request_timeout=2000)
service_context = ServiceContext.from_defaults(llm=llm,embed_model="local")

/tmp/ipykernel_5484/3110638946.py:3: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=llm,embed_model="local")


OutOfMemoryError: CUDA out of memory. Tried to allocate 46.00 MiB. GPU 0 has a total capacity of 3.81 GiB of which 36.38 MiB is free. Process 1871 has 2.59 GiB memory in use. Including non-PyTorch memory, this process has 1.17 GiB memory in use. Of the allocated memory 1.00 GiB is allocated by PyTorch, and 71.74 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [5]:
# Indexing
index = VectorStoreIndex.from_documents(documents,service_context=service_context)

# This sometimes throws an OutOfMemoryError. Solution pending

OutOfMemoryError: CUDA out of memory. Tried to allocate 120.00 MiB. GPU 0 has a total capacity of 3.81 GiB of which 36.38 MiB is free. Process 1871 has 2.59 GiB memory in use. Including non-PyTorch memory, this process has 1.17 GiB memory in use. Of the allocated memory 1.00 GiB is allocated by PyTorch, and 71.74 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Querying
query_engine = index.as_query_engine()

In [ ]:
# Sending the query
response = query_engine.query("The dataset consists of responses to a satisfaction survey. Provide a summary of satisfaction by person assisted by each 'socio implementador u Organización ganadora de Grant Agreement'. Keep it to 50 words per organization.")

In [ ]:
# Printing the response
print(response)

In [ ]:
display(Markdown(f"<b>{response}</b>"))